In [ ]:
pip install owlready2

In [ ]:
pip install mysql-connector-python

In [ ]:
pip install Levenshtein

In [1]:
import sqlite3
import pandas as pd
import csv
import mysql.connector
import numpy as numpy
import json
from sqlite3 import Error
from owlready2 import *
from math import floor, ceil
from Levenshtein import distance as lev
from enum import Enum 

In [2]:
dataname = 'database.db'
memory = ':memory:'
try:
    con = sqlite3.connect(dataname)
    print("Connection is established:" + dataname)
except Error:
    print (Error)
        
def sql_table(con):
    cursorObj = con.cursor()
    sql_file = open("Agencia.sql")
    sql_as_string = sql_file.read()
    cursorObj.executescript(sql_as_string)
    con.commit()

#http://vocab.gtfs.org/gtfs.ttl
sql_table(con)

Connection is established:database.db


OperationalError: table AGENCY already exists

In [3]:
def connect_database():
    try:
        connection = mysql.connector.connect(host='localhost',
                                     user='root',
                                     password='gtfs',
                                     database='gtfs',
                                     charset='utf8mb4')

        if connection.is_connected():
                db_Info = connection.get_server_info()
                print("Connected to MySQL Server version ", db_Info)
                cursor = connection.cursor()
                cursor.execute("select database();")
                record = cursor.fetchone()
                print("You're connected to database: ", record)
            
        return connection, cursor

    except Error as e:
        print("Error while connecting to MySQL", e)
    #finally:
    #    if connection.is_connected():
    #        cursor.close()
    #        connection.close()
    #        print("MySQL connection is closed")

In [4]:
def get_tables(connection, cursor):
    cursor.execute("show tables;")
    myresult = cursor.fetchall()
    return myresult

In [ ]:


cur = connection.cursor()
#cur.execute("SELECT * FROM AGENCY")
#cur.fetchall()
cur.execute('DROP DATABASE IF EXISTS prueba.db')
df = pd.read_sql_query("SELECT * FROM AGENCY",con)
print(df.to_string())



In [ ]:
df = pd.read_csv('examples/datasets/sql/1/AGENCY.csv')
print(df.to_string()) 

In [12]:
def loadOntology(onto):
    my_world = World()
    ontology = onto
    onto = my_world.get_ontology(ontology).load()
    return onto

In [13]:
print(onto.base_iri)
print(onto.name)

http://vocab.gtfs.org/terms#
gtfs


In [6]:
def get_ontology_classes():
    try:
        l = list(my_world.sparql("""
                   SELECT  DISTINCT ?x 
                   { ?x a rdfs:Class . }
            """))
    except:
        l = list(my_world.sparql("""
                   SELECT  DISTINCT ?x 
                   { ?x a owl:Class . }
            """))
    class_list_names = []
    pair_class_names = []
    for elem in l:
        terms = str(elem).split('.')
        pair_class_names.append((re.sub(']', '',terms.pop()), elem))
    return pair_class_names

def get_class_properties(class_prefix):
    try:
        l = list(my_world.sparql("""
                   SELECT  DISTINCT ?p where
                   { """ + class_prefix + """ ?p . }
            """))
    except:
        print("error")
    prp_list_names = []
    pair_prp_names = []
    for elem in l:
        terms = str(elem).split('.')
        pair_prp_names.append((re.sub(']', '',terms.pop()), elem))
    return pair_prp_names

In [11]:
connection, cursor = connect_database()
tables = get_tables(connection,cursor)

for y in tables:
        sql_select_Query = "select * from " + y[0]
        cursor = connection.cursor()
        cursor.execute(sql_select_Query)
        # get all records
        records = cursor.fetchall()
        print("Total number of columns in table: " + y[0], cursor.column_names)
    
print("\nReading  Ontology...\n")
oc = get_ontology_classes()
#oprp = get_ontology_properties()
print(oc)
#print (oprp)
print(tables)



Connected to MySQL Server version  8.0.26
You're connected to database:  ('gtfs',)
Total number of columns in table: AGENCY ('agency_id', 'agency_name', 'agency_url', 'agency_timezone', 'agency_lang', 'agency_phone', 'agency_fare_url')
Total number of columns in table: CALENDAR ('service_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'start_date', 'end_date')
Total number of columns in table: CALENDAR_DATES ('service_id', 'date', 'exception_type')
Total number of columns in table: FEED_INFO ('feed_publisher_name', 'feed_publisher_url', 'feed_lang', 'feed_start_date', 'feed_end_date', 'feed_version')
Total number of columns in table: FREQUENCIES ('trip_id', 'start_time', 'end_time', 'headway_secs', 'exact_times')
Total number of columns in table: ROUTES ('route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color')
Total number of columns in table: SHAPES ('shape_id', 'shape

NameError: name 'my_world' is not defined

In [7]:
def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    return distances[len(token1)][len(token2)]
    
#min(1, 1 - abs(t1-t2)/len(token1))

def string_distance(str1, str2):
    distances = numpy.zeros(len(str1))
    streak = 0
    for x in range(0, len(str1)):
        for y in range(x, len(str2)):
            char1 = str1[x].lower()
            char2 = str2[y].lower()
            if char1 == char2:
                val = min(1, 1 - abs(x-y)/(max(len(str1), len(str2))))
                #print(val)
                if distances[x] < val:
                    distances[x] = val + streak
                if val == 1:
                    streak += 0.1
                    break
    #print(distances)
    return numpy.sum(distances)/(max(len(str1), len(str2)))

def correspondenceClass(ontology, table):
    dis = 10
    term = ""
    for x in ontology:
        #print(x[0])
        term1 = str(table[0]).replace('_', '').lower()
        term2 = str(x[0]).replace('_', '').lower()
        auxDis = levenshteinDistanceDP(term1, term2)
        #print(auxDis)
        if auxDis < dis:
            dis = auxDis
            term = x
    return term
#print("Puntuación: " + str(string_distance('CalendarDates', 'CalendarDateRule')))
for x in range(0, len(tables)):
    print(tables[x])
    print(correspondenceClass(oc, tables[x]))
#print(tables[4][0])
#print(oc[3][0])
#for x in range(0, len(tables)):
#    print("Puntuación: " + tables[0][0].lower() + " " + oc[x][0].lower() + " :" + str(lev(tables[0][0].lower(), oc[x][0].lower())))

NameError: name 'tables' is not defined

In [10]:
def loadProperties(outputFile, propertiesFile):
    with open('properties/' + propertiesFile) as json_file:
        data = json.load(json_file)
        json_file.close()
    return data

def writeProperties(output, properties):
    with open('outputs/' + output, 'w') as fw:
        original_stdout = sys.stdout
        sys.stdout = fw
        for x in properties['list_prefixes']:
            print('@prefix ' + x['prefix'] + ": " + x['URI'])
        print('@base ' + properties['base']['URI'])
        sys.stdout = original_stdout
        fw.close()
#print(data['list_prefixes'])

def LogicalSource(table, SQLversion):
    level = 8
    print (" "*level, "rml:logicalSource [")
    print (" "*(2*level), "rml:source: " + '"' + "<#DB_source>" + '"')
    print (" "*(2*level), "rr:sqlVersion " + '"' + SQLversion + '"')
    print (" "*(2*level), "rr:tableName: " + '"' + table[0] + '"')
    print (" "*level, "];")
    
def TripleMap(subject):
    level = 8
    print ("<" + subject[0] + ">")
    print (" "*level, "a rr:TriplesMap;")
    #print ("\n")
    LogicalSource(subject, "SQL2008")
    
#def SubjectMap(ontologyClasses):
    

def deleteFileContent(file):
    file = open("outputs/" + file,"r+")
    file.truncate(0)
    file.close()
#def RMLTable(oc, table):

def RML_Transformation(output, properties, tables, ontologyClasses):
    data = loadProperties(output, properties)
    writeProperties(output, data)
    with open('outputs/' + output, 'w') as fw:
        original_stdout = sys.stdout
        sys.stdout = fw
        for x in tables:
            TripleMap(x)
        sys.stdout = original_stdout
        fw.close()
        
connection, cursor = connect_database()
tables = get_tables(connection,cursor)
onto = loadOntology("gtfs")
ontologyClasses = get_ontology_classes()
RML_Transformation('output.txt', 'properties.json', tables, ontologyClasses)

Connected to MySQL Server version  8.0.26
You're connected to database:  ('gtfs',)


NameError: name 'my_world' is not defined